# Tutorial 03: Multi-Turn Conversations - Stateful Travel Assistant

##  Learning Objectives
By the end of this notebook, you will:
- Understand what threads are and why they're essential
- Create stateful conversations that remember context
- Manage conversation threads with Azure AI
- Build natural, flowing dialogues with your travel assistant
- Learn when to create new threads vs. reusing existing ones

##  Key Concepts

### What is a Thread?
A **thread** is a conversation history that maintains context across multiple queries.

**Without threads:**
```
User: "I want to visit Japan"
Agent: "Great! Japan has many attractions..."

User: "What's the weather there?"
Agent: "Where would you like weather info?"  Forgot Japan!
```

**With threads:**
```
User: "I want to visit Japan"
Agent: "Great! Japan has many attractions..."

User: "What's the weather there?"
Agent: "Let me check the weather in Japan..."  Remembers!
```

### Thread Types

1. **Automatic Threads** (stateless)
   - New thread created for each `run()` call
   - No memory between queries
   - Good for: One-off questions

2. **Persistent Threads** (stateful)
   - Same thread reused across multiple calls
   - Maintains full conversation history
   - Good for: Multi-turn conversations

---

## Step 1: Setup and Imports

In [1]:
import asyncio
import os
from random import randint, choice
from typing import Annotated

from agent_framework import ChatAgent, AgentThread
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from pydantic import Field
from dotenv import load_dotenv

load_dotenv()
print(" Imports successful!")

 Imports successful!


## Step 2: Define Our Travel Tools

Let's create tools that our agent will use throughout conversations.

In [2]:
def get_weather(
    location: Annotated[str, Field(description="City or country name")],
) -> str:
    """Get current weather for a location."""
    conditions = ["sunny", "partly cloudy", "cloudy", "rainy", "windy"]
    temp = randint(15, 32)
    condition = choice(conditions)
    return f"Weather in {location}: {condition}, {temp}°C"

def get_attractions(
    location: Annotated[str, Field(description="City or country name")],
) -> str:
    """Get top attractions for a destination."""
    # Simulated attraction data
    attractions = {
        "Japan": "Tokyo Tower, Mount Fuji, Kyoto Temples, Osaka Castle",
        "Paris": "Eiffel Tower, Louvre Museum, Notre-Dame, Arc de Triomphe",
        "London": "Big Ben, Tower of London, British Museum, Buckingham Palace",
        "Barcelona": "Sagrada Familia, Park Güell, La Rambla, Gothic Quarter",
    }
    
    for city, attrs in attractions.items():
        if city.lower() in location.lower():
            return f"Top attractions in {location}: {attrs}"
    
    return f"Popular attractions in {location}: Historic sites, museums, local markets, parks"

def estimate_budget(
    destination: Annotated[str, Field(description="Destination city or country")],
    days: Annotated[int, Field(description="Number of days")],
) -> str:
    """Estimate travel budget for a destination."""
    daily_costs = {
        "Japan": 150,
        "Paris": 180,
        "London": 200,
        "Barcelona": 120,
        "Thailand": 60,
    }
    
    cost_per_day = daily_costs.get(destination, 100)
    total = cost_per_day * days
    
    return f"Estimated budget for {days} days in {destination}: ${total} (${cost_per_day}/day)"

print(" Tools defined")

 Tools defined


## Step 3: The Problem - No Memory (Stateless)

Let's see what happens WITHOUT threads - each query is independent.

In [4]:
async def example_without_threads():
    """
    This shows the problem: agent doesn't remember previous context.
    Each run() creates a new thread automatically.
    """
    print("=== Without Threads (No Memory) ===")
    print("Each query is independent - agent forgets previous conversation\n")
    
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(async_credential=credential),
            name="StatelessTravelAgent",
            instructions="You are a helpful travel assistant.",
            tools=[get_weather, get_attractions, estimate_budget]
        ) as agent,
    ):
        # First query
        query1 = "I'm planning a trip to Japan next month."
        print(f"User: {query1}")
        response1 = await agent.run(query1)
        print(f"Agent: {response1.text}\n")
        
        # Second query - expects agent to remember Japan
        query2 = "What's the weather like there?"
        print(f"User: {query2}")
        response2 = await agent.run(query2)
        print(f"Agent: {response2.text}\n")
        
        print(" Notice: Agent doesn't know where 'there' is!")
        print("   Each run() created a separate thread.\n")

await example_without_threads()

=== Without Threads (No Memory) ===
Each query is independent - agent forgets previous conversation

User: I'm planning a trip to Japan next month.
Agent: That sounds exciting! Japan is a fantastic destination with a mix of rich culture, beautiful nature, and modern attractions.

To better assist you, could you let me know:
- Which cities or regions in Japan you plan to visit?
- How long you’ll be staying?
- What kind of experiences you’re interested in (e.g., food, culture, nature, shopping, anime, historical sites)?
- Your approximate budget (if you want planning tips based around it)?

I can help with suggested itineraries, must-see attractions, travel tips, estimated budgets, and anything else you need for your trip. Just let me know your preferences!

User: What's the weather like there?
Agent: Could you please specify which location you are asking about? I’ll be happy to provide the current weather for that place!

 Notice: Agent doesn't know where 'there' is!
   Each run() creat

## Step 4: The Solution - Persistent Threads

Now let's use a **persistent thread** to maintain conversation context.

In [5]:
async def example_with_thread():
    """
    This shows the solution: using the same thread maintains context.
    """
    print("=== With Persistent Thread (Memory) ===")
    print("Same thread maintains context across multiple queries\n")
    
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(async_credential=credential),
            name="StatefulTravelAgent",
            instructions="You are a helpful travel assistant.",
            tools=[get_weather, get_attractions, estimate_budget]
        ) as agent,
    ):
        # Create a new thread that we'll reuse
        thread = agent.get_new_thread()
        
        # First query
        query1 = "I'm planning a trip to Japan next month."
        print(f"User: {query1}")
        response1 = await agent.run(query1, thread=thread)
        print(f"Agent: {response1.text}\n")
        
        # Second query - agent remembers Japan!
        query2 = "What's the weather like there?"
        print(f"User: {query2}")
        response2 = await agent.run(query2, thread=thread)
        print(f"Agent: {response2.text}\n")
        
        # Third query - agent remembers the whole conversation
        query3 = "What are the top attractions I should visit?"
        print(f"User: {query3}")
        response3 = await agent.run(query3, thread=thread)
        print(f"Agent: {response3.text}\n")
        
        print(" Success: Agent remembers the entire conversation!")
        print(f"   Thread ID: {thread.service_thread_id}\n")

await example_with_thread()

=== With Persistent Thread (Memory) ===
Same thread maintains context across multiple queries

User: I'm planning a trip to Japan next month.
Agent: That sounds exciting! Japan is a beautiful and diverse country with so much to offer. To help you plan the perfect trip, could you please provide more details, such as:

- Which cities or regions in Japan do you want to visit?
- How many days do you plan to stay?
- What are your interests? (e.g., culture, food, nature, shopping, anime, history, etc.)
- Are you traveling solo, as a couple, with family, or with friends?
- Is there a specific budget you'd like to stick to?

Let me know a bit more, and I can suggest an itinerary, must-see attractions, budget estimates, and travel tips!

User: What's the weather like there?
Agent: The current weather in Japan is rainy with a temperature of 28°C. In general, June in Japan is part of the rainy season (called "tsuyu"), so you can expect periodic showers and warm, humid weather.

If you let me know

## Step 5: Natural Multi-Turn Planning Conversation

Let's have a realistic trip planning conversation with multiple back-and-forth exchanges.

In [7]:
async def planning_conversation():
    """
    A natural conversation where the agent helps plan a trip.
    """
    print("=== Natural Trip Planning Conversation ===")
    print("Multi-turn dialogue with context awareness\n")
    
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(async_credential=credential),
            name="TravelPlanningAgent",
            instructions="""
            You are an expert travel planner. Help users plan their trips by:
            - Asking clarifying questions when needed
            - Remembering all details they share
            - Using tools to provide accurate information
            - Being enthusiastic and helpful
            """,
            tools=[get_weather, get_attractions, estimate_budget]
        ) as agent,
    ):
        thread = agent.get_new_thread()
        
        # Conversation flow
        queries = [
            "I have 2 weeks off in December and want to go somewhere warm.",
            "I'm thinking either Thailand or Barcelona. What do you recommend?",
            "Let's go with Barcelona! What's the weather like in December?",
            "How much should I budget for 7 days?",
            "That works! What are the must-see attractions?",
        ]
        
        for i, query in enumerate(queries, 1):
            print(f"{'='*60}")
            print(f"Turn {i}")
            print(f"{'='*60}")
            print(f"User: {query}\n")
            
            response = await agent.run(query, thread=thread)
            print(f"Agent: {response.text}\n")
        
        print(" Complete conversation with full context maintained!")
        print(f"   Thread ID: {thread.service_thread_id}\n")

await planning_conversation()

=== Natural Trip Planning Conversation ===
Multi-turn dialogue with context awareness

Turn 1
User: I have 2 weeks off in December and want to go somewhere warm.

Agent: That sounds fantastic! December is a great time to escape the cold and enjoy some sunshine. To help you find the perfect destination, could you share a few more details?

1. **Where are you starting from?** (city/country)
2. **What’s your travel style?** (adventure, relaxation, culture, beaches, etc.)
3. **Any preferences on budget?** (luxury, mid-range, budget)
4. **Do you want to stay in one place or visit multiple destinations?**
5. **Are you traveling solo, with friends, family, or a partner?**

Once I have these details, I can recommend warm destinations and help you plan an amazing trip!

Turn 2
User: I'm thinking either Thailand or Barcelona. What do you recommend?

Agent: That sounds fantastic! December is a great time to escape the cold and enjoy some sunshine. To help you find the perfect destination, could y

## Step 6: Multiple Conversations with Different Threads

You can manage multiple conversations simultaneously using different threads.

In [8]:
async def multiple_conversations():
    """
    Manage multiple separate conversations with different threads.
    """
    print("=== Multiple Conversations (Different Threads) ===")
    print("Each thread maintains its own separate context\n")
    
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(async_credential=credential),
            name="MultiConversationAgent",
            instructions="You are a helpful travel assistant.",
            tools=[get_weather, get_attractions]
        ) as agent,
    ):
        # Create two separate threads for two different users/conversations
        alice_thread = agent.get_new_thread()
        bob_thread = agent.get_new_thread()
        
        # Alice's conversation about Paris
        print("🧑 Alice's Conversation:")
        alice_q1 = "I'm interested in visiting Paris."
        print(f"Alice: {alice_q1}")
        response = await agent.run(alice_q1, thread=alice_thread)
        print(f"Agent: {response.text[:100]}...\n")
        
        # Bob's conversation about London
        print("👨 Bob's Conversation:")
        bob_q1 = "I'm thinking about London for my next trip."
        print(f"Bob: {bob_q1}")
        response = await agent.run(bob_q1, thread=bob_thread)
        print(f"Agent: {response.text[:100]}...\n")
        
        # Continue Alice's conversation
        print("🧑 Alice continues:")
        alice_q2 = "What's the weather there?"
        print(f"Alice: {alice_q2}")
        response = await agent.run(alice_q2, thread=alice_thread)
        print(f"Agent: {response.text}\n")
        
        # Continue Bob's conversation
        print("👨 Bob continues:")
        bob_q2 = "What attractions should I visit there?"
        print(f"Bob: {bob_q2}")
        response = await agent.run(bob_q2, thread=bob_thread)
        print(f"Agent: {response.text}\n")
        
        print(" Both conversations maintained separate context!")
        print(f"   Alice's thread: {alice_thread.service_thread_id}")
        print(f"   Bob's thread: {bob_thread.service_thread_id}\n")

await multiple_conversations()

=== Multiple Conversations (Different Threads) ===
Each thread maintains its own separate context

🧑 Alice's Conversation:
Alice: I'm interested in visiting Paris.
Agent: Paris is a fantastic destination! Here’s what you need to know right now:

Top Attractions:
- Eiffel...

👨 Bob's Conversation:
Bob: I'm thinking about London for my next trip.
Agent: Paris is a fantastic destination! Here’s what you need to know right now:

Top Attractions:
- Eiffel...

👨 Bob's Conversation:
Bob: I'm thinking about London for my next trip.
Agent: London is a fantastic choice for a trip! It’s a city full of history, culture, amazing food, and ple...

🧑 Alice continues:
Alice: What's the weather there?
Agent: London is a fantastic choice for a trip! It’s a city full of history, culture, amazing food, and ple...

🧑 Alice continues:
Alice: What's the weather there?
Agent: The current weather in Paris is partly cloudy with a temperature of 30°C. Let me know if you need more details or a forecast for upcomi

## Step 7: Resuming Conversations with Thread IDs

You can save the thread ID and resume conversations later.

In [9]:
async def resume_conversation():
    """
    Save a thread ID and resume the conversation later.
    """
    print("=== Resuming Conversations with Thread IDs ===")
    
    saved_thread_id = None
    
    # Part 1: Start a conversation and save the thread ID
    print("\n--- Part 1: Starting Conversation ---")
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(async_credential=credential),
            name="ResumableAgent",
            instructions="You are a helpful travel assistant.",
            tools=[get_weather, estimate_budget]
        ) as agent,
    ):
        thread = agent.get_new_thread()
        
        query = "I want to visit Barcelona for 5 days. What's the budget?"
        print(f"User: {query}")
        response = await agent.run(query, thread=thread)
        print(f"Agent: {response.text}\n")
        
        # Save the thread ID for later
        saved_thread_id = thread.service_thread_id
        print(f" Saved thread ID: {saved_thread_id}")
        print("   (In a real app, you'd save this to a database)\n")
    
    # Part 2: Resume the conversation using the saved thread ID
    print("--- Part 2: Resuming Conversation Later ---")
    if saved_thread_id:
        async with (
            AzureCliCredential() as credential,
            ChatAgent(
                chat_client=AzureAIAgentClient(
                    async_credential=credential,
                    thread_id=saved_thread_id  # Resume with this thread
                ),
                name="ResumableAgent",
                instructions="You are a helpful travel assistant.",
                tools=[get_weather, estimate_budget]
            ) as agent,
        ):
            # Create thread object with the saved ID
            thread = AgentThread(service_thread_id=saved_thread_id)
            
            query = "What's the weather like there?"
            print(f"User: {query}")
            response = await agent.run(query, thread=thread)
            print(f"Agent: {response.text}\n")
            
            print(" Successfully resumed conversation from saved thread ID!")
            print("   Agent remembered Barcelona from the previous session.\n")

await resume_conversation()

=== Resuming Conversations with Thread IDs ===

--- Part 1: Starting Conversation ---
User: I want to visit Barcelona for 5 days. What's the budget?
Agent: The estimated budget for a 5-day trip to Barcelona is $600, which averages about $120 per day. This typically covers accommodation, meals, transportation, and some sightseeing. Let me know if you need a more detailed breakdown or specific travel tips!

 Saved thread ID: thread_2dnm4uPqcmRNugeN5MHU2JOQ
   (In a real app, you'd save this to a database)

Agent: The estimated budget for a 5-day trip to Barcelona is $600, which averages about $120 per day. This typically covers accommodation, meals, transportation, and some sightseeing. Let me know if you need a more detailed breakdown or specific travel tips!

 Saved thread ID: thread_2dnm4uPqcmRNugeN5MHU2JOQ
   (In a real app, you'd save this to a database)

--- Part 2: Resuming Conversation Later ---
User: What's the weather like there?
--- Part 2: Resuming Conversation Later ---
User

## Step 8: Inspecting Thread Messages

You can examine the conversation history stored in a thread.

In [10]:
async def inspect_thread():
    """
    Examine the messages and history in a thread.
    Note: With Azure AI service-managed threads, messages are stored in Azure.
    """
    print("=== Inspecting Thread Messages ===")
    
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(async_credential=credential),
            name="InspectableAgent",
            instructions="You are a helpful travel assistant.",
            tools=[get_weather]
        ) as agent,
    ):
        thread = agent.get_new_thread()
        
        # Have a short conversation
        queries = [
            "What's the weather in Tokyo?",
            "How about London?",
            "Which has better weather?"
        ]
        
        print("Having a conversation...\n")
        for i, query in enumerate(queries, 1):
            print(f"Query {i}: {query}")
            response = await agent.run(query, thread=thread)
            print(f"Response: {response.text[:100]}...\n")
        
        # Inspect the thread
        print("="*60)
        print(f"Thread ID: {thread.service_thread_id}")
        print(f"Thread is service-managed: {thread.service_thread_id is not None}")
        print(f"Local message store: {thread.message_store}")
        print("="*60)
        print("\nNote: With Azure AI, messages are stored in the service.")
        print("The thread ID is used to access the conversation history in Azure.")

await inspect_thread()

=== Inspecting Thread Messages ===
Having a conversation...

Query 1: What's the weather in Tokyo?
Response: The current weather in Tokyo is sunny with a temperature of 24°C. Let me know if you need more detai...

Query 2: How about London?
Response: The current weather in Tokyo is sunny with a temperature of 24°C. Let me know if you need more detai...

Query 2: How about London?
Response: The current weather in London is sunny with a temperature of 27°C. If you'd like more information or...

Query 3: Which has better weather?
Response: The current weather in London is sunny with a temperature of 27°C. If you'd like more information or...

Query 3: Which has better weather?
Response: Both Tokyo and London currently have sunny weather, which is generally considered very pleasant. How...

Thread ID: thread_qhwCTPjZdLC88UHYQZwe2X3X
Thread is service-managed: True
Local message store: None

Note: With Azure AI, messages are stored in the service.
The thread ID is used to access the convers

##  Understanding Thread Lifecycle

### Thread Creation and Management

```python
# Option 1: Let Azure create and manage threads automatically
response = await agent.run(query)  # New thread each time

# Option 2: Create and reuse a thread (recommended)
thread = agent.get_new_thread()
response = await agent.run(query, thread=thread)  # Same thread

# Option 3: Resume from a saved thread ID
thread = AgentThread(service_thread_id=saved_id)
response = await agent.run(query, thread=thread)
```

### Thread Storage in Azure

- Threads are **stored in Azure AI Foundry**
- Thread IDs are **persistent** across sessions
- You can **resume conversations** anytime
- Azure manages **cleanup and retention**

### When to Use Threads

| Use Case | Thread Strategy |
|----------|----------------|
| One-off questions | No thread (automatic) |
| Chat conversation | Single persistent thread |
| Multi-user app | One thread per user |
| Session-based | New thread per session |
| Long-term assistant | Save & resume thread IDs |

## 🗑 Agent Lifecycle Management in Azure AI Foundry

### Automatic Agent Cleanup

**Important:** In Azure AI Foundry, **agents are automatically deleted** after execution completes. This is a key architectural difference from other platforms.

#### How It Works

```python
async with (
    AzureCliCredential() as credential,
    ChatAgent(
        chat_client=AzureAIAgentClient(async_credential=credential),
        name="TravelAgent",
        instructions="You are a travel assistant.",
    ) as agent,  # ← Agent is created here
):
    # Agent exists in Azure during this block
    response = await agent.run("Hello")
    
# ← Agent is DELETED from Azure here (when context exits)
```

**What happens:**
1. **Agent Created** - When entering the `async with` block, agent is created in Azure AI Foundry
2. **Agent Used** - You can run queries, create threads, use tools
3. **Agent Deleted** - When exiting the block, agent definition is removed from Azure
4. **Threads Persist** - Thread data remains in Azure (only agent definition is deleted)

### What Gets Deleted vs. What Persists

| Resource | After Context Exit | Why |
|----------|-------------------|-----|
| **Agent Definition** |  Deleted | Reduces clutter, saves costs |
| **Agent Name** |  Deleted | Agent resource is removed |
| **Agent Instructions** |  Deleted | Part of agent definition |
| **Agent Tools** |  Deleted | Part of agent definition |
| **Thread ID** |  Persists | Conversation history preserved |
| **Thread Messages** |  Persists | Your data is kept |
| **Model Deployment** |  Persists | Shared resource |

### Advantages of Automatic Deletion

#### 1. **Cost Efficiency** 
```python
# Traditional approach (agent persists)
agent_id = "agent-123"  # Agent remains in Azure, costs accumulate
# You pay for agent storage even when not using it

# Azure AI Foundry approach (ephemeral agents)
async with ChatAgent(...) as agent:
    # Agent exists only when needed
    await agent.run(query)
# Agent deleted, no ongoing costs for unused agents
```

**Benefit:** Pay only for compute during execution, not for idle agent storage

#### 2. **Resource Management** 🧹
```python
# Problem with persistent agents:
# - Accumulation of old/test agents
# - Namespace pollution
# - Manual cleanup required

# Azure AI Foundry solution:
async with ChatAgent(name="TestAgent") as agent:
    # Create and test
    pass
# Auto-cleanup - no leftover test agents!
```

**Benefit:** No "agent sprawl" - your Azure workspace stays clean

#### 3. **Security & Compliance** 🔒
```python
# Sensitive agent with PII access
async with ChatAgent(
    name="CustomerDataAgent",
    instructions="Access customer PII for support"
) as agent:
    # Agent exists briefly
    response = await agent.run("Get user info")
# Agent definition deleted, reduces attack surface
```

**Benefit:** Shorter-lived credentials, reduced security risk

#### 4. **Version Control & GitOps** 
```python
# Agent definition in your code
async with ChatAgent(
    name="V2TravelAgent",  # Version in code
    instructions="Updated instructions..."  # Source controlled
) as agent:
    await agent.run(query)
```

**Benefit:** Agent behavior is code-defined, not state in Azure

### Disadvantages & Tradeoffs

| Disadvantage | Mitigation |
|--------------|------------|
| **Agent recreated each run** | Minimal overhead (~100-300ms) |
| **Can't view agents in portal** | View threads/messages instead |
| **Requires code to define** | Better for version control |
| **No long-running agents** | Use threads for continuity |

### Alternative: Persistent Agents (Other Platforms)

Some platforms keep agents persistent:

```python
# Hypothetical persistent agent approach
agent = create_agent(name="MyAgent")  # Agent stays in platform
agent_id = agent.id  # "agent-abc123"

# Later (different session)
agent = get_agent(agent_id)  # Retrieve existing agent
response = agent.run("Hello")  # Use persistent agent
```

**When persistent agents make sense:**
- Agents with complex, evolving instructions
- Shared agents across multiple services
- Agents that learn/adapt over time
- Long-running background agents

**Azure AI Foundry's philosophy:**
- Agents are **code** (in your repository)
- Threads are **data** (in Azure)
- Separates compute (ephemeral) from state (persistent)

### Best Practices for Azure AI Foundry

#### 1. **Define Agents as Code**
```python
# Good: Agent configuration in code
def create_travel_agent(credential):
    return ChatAgent(
        chat_client=AzureAIAgentClient(async_credential=credential),
        name="TravelAgent",
        instructions="You are a helpful travel assistant...",
        tools=[get_weather, get_attractions]
    )

# Use it
async with create_travel_agent(credential) as agent:
    response = await agent.run(query, thread=thread)
```

#### 2. **Persist Thread IDs, Not Agent IDs**
```python
# Save to database
session_data = {
    "user_id": user.id,
    "thread_id": thread.service_thread_id,  #  Save this
    # "agent_id": agent.id  #  Don't do this (agent will be deleted)
}

# Restore later
async with create_travel_agent(credential) as agent:  # New agent
    thread = AgentThread(service_thread_id=session_data["thread_id"])  # Same thread
    response = await agent.run(query, thread=thread)
```

#### 3. **Use Context Managers**
```python
#  Good: Automatic cleanup
async with ChatAgent(...) as agent:
    await agent.run(query)

#  Avoid: Manual management
agent = ChatAgent(...)
await agent.run(query)
# Agent might not be cleaned up properly
```

### Comparison Table: Azure AI vs Traditional

| Aspect | Azure AI Foundry | Traditional Persistent |
|--------|------------------|----------------------|
| Agent Lifetime | Ephemeral (seconds) | Persistent (days/months) |
| Storage Cost | None for agents | Ongoing for agents |
| Cleanup | Automatic | Manual required |
| Version Control | In code (Git) | In platform (API) |
| Scaling | Infinite agents | Limited by quota |
| Thread Persistence |  Yes |  Yes |
| Agent Evolution | Code updates | Platform updates |
| Multi-tenancy | Isolated per execution | Shared agents |

## 🔬 Practical Example: Agent Lifecycle Demo

Let's see the ephemeral agent behavior in action:

In [ ]:
async def agent_lifecycle_demo():
    """
    Demonstrates ephemeral agents with persistent threads.
    Shows that thread data survives even though agents are deleted.
    """
    print("=== Agent Lifecycle Demonstration ===\n")
    
    saved_thread_id = None
    
    # Session 1: Create agent, have conversation, agent gets deleted
    print("--- Session 1: Creating Agent and Thread ---")
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(async_credential=credential),
            name="EphemeralAgent_Session1",
            instructions="You are a helpful assistant. Remember what users tell you.",
            tools=[get_weather]
        ) as agent,
    ):
        print(f" Agent created: {agent.name}")
        
        thread = agent.get_new_thread()
        saved_thread_id = thread.service_thread_id
        print(f" Thread created: {saved_thread_id}\n")
        
        query1 = "My favorite city is Barcelona."
        print(f"User: {query1}")
        response1 = await agent.run(query1, thread=thread)
        print(f"Agent: {response1.text}\n")
        
        print(f" Saving thread ID for later: {saved_thread_id}")
    
    print("\n🗑  Agent 'EphemeralAgent_Session1' has been DELETED from Azure")
    print("    (automatically when exiting the 'async with' block)")
    print("📦 Thread data still exists in Azure AI Foundry\n")
    
    # Session 2: New agent, same thread - conversation continues!
    print("--- Session 2: New Agent, Same Thread ---")
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(async_credential=credential),
            name="EphemeralAgent_Session2",  # Different agent!
            instructions="You are a helpful assistant. Remember what users tell you.",
            tools=[get_weather]
        ) as agent,
    ):
        print(f" New agent created: {agent.name}")
        print(f" Resuming thread: {saved_thread_id}\n")
        
        # Resume the same thread
        thread = AgentThread(service_thread_id=saved_thread_id)
        
        query2 = "What's the weather in my favorite city?"
        print(f"User: {query2}")
        response2 = await agent.run(query2, thread=thread)
        print(f"Agent: {response2.text}\n")
        
        print(" Agent remembered Barcelona from previous session!")
        print("   (Even though it's a completely different agent)")
    
    print("\n🗑  Agent 'EphemeralAgent_Session2' has been DELETED from Azure\n")
    
    print("="*60)
    print("KEY INSIGHTS:")
    print("="*60)
    print("1. Agents are ephemeral - created and deleted each session")
    print("2. Threads are persistent - conversation history survives")
    print("3. Different agents can continue the same conversation")
    print("4. Thread ID is the key to conversation continuity")
    print("5. Agent definitions live in your code, not in Azure")
    print("="*60)

await agent_lifecycle_demo()

##  Key Takeaways

### Thread Benefits
1. **Context Awareness** - Agent remembers previous exchanges
2. **Natural Conversations** - Users don't need to repeat information
3. **Better UX** - Feels like talking to a human
4. **Stateful Interactions** - Build on previous responses

### Best Practices
1. **Use threads for conversations** - Always use them for multi-turn dialogues
2. **Save thread IDs** - Store them in your database for users
3. **One thread per conversation** - Don't mix different topics
4. **Clean up old threads** - Delete when conversations end (Azure helps with this)

### Common Patterns

**Web Chat Application:**
```python
# Store thread_id in user session
if not session.get('thread_id'):
    thread = agent.get_new_thread()
    session['thread_id'] = thread.service_thread_id
else:
    thread = AgentThread(service_thread_id=session['thread_id'])
```

**Customer Support:**
```python
# One thread per support ticket
thread_id = ticket.get_thread_id()
thread = AgentThread(service_thread_id=thread_id)
```

##  Practice Exercises

1. **Create a booking conversation** - Plan a complete trip from start to booking
2. **Compare destinations** - Have agent remember and compare multiple cities
3. **Build a preference profile** - Agent learns user preferences over conversation
4. **Implement thread management** - Save/load thread IDs from a dictionary

In [ ]:
# Exercise: Create a conversation where the agent helps compare 3 destinations
# and remembers all the details discussed about each one

async def compare_destinations_exercise():
    """
    Your turn! Create a conversation that:
    1. Discusses Paris (weather, attractions, budget)
    2. Discusses Tokyo (same info)
    3. Discusses Barcelona (same info)
    4. Asks agent to compare all three and recommend
    
    The agent should remember ALL details from the conversation!
    """
    # Your code here
    pass

# Uncomment to test your solution
# await compare_destinations_exercise()

##  What's Next?

Great work! You now understand how to create stateful, context-aware conversations.

But our agent still has limitations:
-  Doesn't remember user preferences across sessions
-  Can't store long-term knowledge about the user
-  Conversation history in threads can get very long

**In Tutorial 04: Context and Memory**, you'll learn to:
- Add persistent memory to your agents
- Store user preferences and profiles
- Use context providers for smarter agents
- Manage conversation context efficiently

---

### Quick Reference

**Create a persistent thread:**
```python
thread = agent.get_new_thread()
response = await agent.run(query, thread=thread)
```

**Save and resume:**
```python
# Save
thread_id = thread.service_thread_id

# Resume
thread = AgentThread(service_thread_id=thread_id)
```

**Multiple conversations:**
```python
alice_thread = agent.get_new_thread()
bob_thread = agent.get_new_thread()
```